In [9]:
from flask import Flask, request, send_file
from flask_socketio import SocketIO, emit
from flask_cors import CORS
import base64
import cv2
import numpy as np
import mediapipe as mp
import io
import os
import math
from tensorflow.keras.models import load_model
import time

app = Flask(__name__)
CORS(app, resources={r"/epa/upload": {"origins": "http://localhost:8080"}}, supports_credentials=True)
socketio = SocketIO(app, cors_allowed_origins="http://localhost:8080")

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

font_scale = 0.5
font_thickness = 1

save_path = "C:/Users/smart/Flask"
model_path = "C:/Users/smart/Flask/epa_model2.hdf5"
epa_model = load_model(model_path)

target_name = ['푸쉬업', '스탠딩 사이드 크런치', '스텝 포워드 다이나믹 런지']

def calculate_angle(point1, point2, point3):
    angle_radians = math.atan2(point3.y - point2.y, point3.x - point2.x) - math.atan2(point1.y - point2.y, point1.x - point2.x)
    angle_degrees = math.degrees(angle_radians)
    return angle_degrees

def check_side_crunch_angle(landmarks):
    left_knee = landmarks.landmark[25]
    left_hip = landmarks.landmark[23]
    left_ankle = landmarks.landmark[27]
    right_knee = landmarks.landmark[26]
    right_hip = landmarks.landmark[24]
    right_ankle = landmarks.landmark[28]
    
    angle_left2 = calculate_angle(left_hip, left_knee, left_ankle)
    angle_right2 = calculate_angle(right_hip, right_knee, right_ankle)
    
    return 70 <= angle_left2 <= 130 or 70 <= angle_right2 <= 130

def check_lunge_angle(landmarks):
    left_knee = landmarks.landmark[25]
    left_hip = landmarks.landmark[23]
    left_ankle = landmarks.landmark[27]
    right_knee = landmarks.landmark[26]
    right_hip = landmarks.landmark[24]
    right_ankle = landmarks.landmark[28]
    
    angle_left = calculate_angle(left_hip, left_knee, left_ankle)
    angle_right = calculate_angle(right_hip, right_knee, right_ankle)
    
    return 70 <= angle_left <= 130 and 70 <= angle_right <= 130

@app.route('/epa/upload', methods=['POST'])
def upload_image():
    if 'image' in request.files:
        image = request.files['image']
        image_np = np.frombuffer(image.read(), np.uint8)
        image = cv2.imdecode(image_np, cv2.IMREAD_COLOR)

        with mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose:
            image_height, image_width, _ = image.shape
            results = pose.process(image)

            if results.pose_landmarks:
                image_copy = image.copy()
                image_copy.flags.writeable = True

                coordinates_for_prediction = []
                for i in [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]:
                    landmark = results.pose_landmarks.landmark[i]
                    x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
                    coordinates_for_prediction.extend([x, y])
                    cv2.circle(image_copy, (x, y), 1, (0, 0, 255), 5)

                for connection in mp_pose.POSE_CONNECTIONS:
                    start_point = connection[0]
                    end_point = connection[1]
                    start_landmark = results.pose_landmarks.landmark[start_point]
                    end_landmark = results.pose_landmarks.landmark[end_point]
                    start_point = (int(start_landmark.x * image_width), int(start_landmark.y * image_height))
                    end_point = (int(end_landmark.x * image_width), int(end_landmark.y * image_height))
                    cv2.line(image_copy, start_point, end_point, (255, 255, 255), 2)

                input_data = [coordinates_for_prediction]
                predict = epa_model.predict(np.array(input_data))
                result = np.argmax(predict)
                confidence = predict[0][result] * 100

                print('결과:', target_name[result])
                print(f'정확도: {confidence:.2f}%')

                if target_name[result] == '스탠딩 사이드 크런치' and confidence >= 70 and check_side_crunch_angle(results.pose_landmarks):
                    detected_exercise = 'standing side crunch'
                    print(check_side_crunch_angle)
                    print('Good!')
                    socketio.emit('exercise_result', {'message': 'Good!', 'exercise': detected_exercise})
                elif target_name[result] == '푸쉬업' and confidence >= 60:
                    detected_exercise = 'pushup'
                    print('Good!')
                    socketio.emit('exercise_result1', {'message': 'Good!', 'exercise': detected_exercise})
                elif target_name[result] == '스텝 포워드 다이나믹 런지' and confidence >= 50 and check_lunge_angle(results.pose_landmarks):
                    detected_exercise = 'lunge'
                    print('Good!')
                    socketio.emit('exercise_result2', {'message': 'Good!', 'exercise': detected_exercise})

                
                save_image_path = os.path.join(os.path.join("C:/Users/smart/Flask", "processed_image.png"))
                cv2.imwrite(save_image_path, image_copy)

                _, img_encoded = cv2.imencode('.png', image_copy)
                img_base64 = base64.b64encode(img_encoded).decode('utf-8')

                socketio.emit('processed_image', {'image': img_base64})
                return send_file(save_image_path, mimetype='image/png')

            else:
                print("Pose detection failed.")
                
                _, img_encoded = cv2.imencode('.png', image)
                img_base64 = base64.b64encode(img_encoded).decode('utf-8')
                socketio.emit('processed_image', {'image': img_base64})
                return send_file(io.BytesIO(image_np), mimetype='image/png')

    return 'Image processing completed.'

@socketio.on('connect')
def handle_connect():
    print('WebSocket connection opened.')

@socketio.on('message')
def handle_message(message):
    print('Message received from the client:', message)

@socketio.on('disconnect')
def handle_disconnect():
    print('WebSocket connection closed.')

if __name__ == '__main__':
    socketio.run(app, host="localhost", port=5000, allow_unsafe_werkzeug=True)

Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Mar/2024 16:34:05] "GET /socket.io/?EIO=4&transport=polling&t=OuDYAoT HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2024 16:34:05] "POST /socket.io/?EIO=4&transport=polling&t=OuDYAtW&sid=hmYW2rnz2wQqOxyzAAAA HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2024 16:34:05] "GET /socket.io/?EIO=4&transport=polling&t=OuDYAtX&sid=hmYW2rnz2wQqOxyzAAAA HTTP/1.1" 200 -


WebSocket connection opened.
1/1 [==============================] - 0s 62ms/step


127.0.0.1 - - [05/Mar/2024 16:34:06] "POST /epa/upload HTTP/1.1" 200 -


...

결과: 스텝 포워드 다이나믹 런지
정확도: 100.00%
Pose detection failed.


127.0.0.1 - - [05/Mar/2024 16:36:21] "POST /epa/upload HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2024 16:36:21] "POST /epa/upload HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2024 16:36:21] "GET /socket.io/?EIO=4&transport=websocket&sid=VLUvMrEKpEwoKsVkAAAE HTTP/1.1" 200 -


Pose detection failed.
Pose detection failed.
WebSocket connection closed.


127.0.0.1 - - [05/Mar/2024 16:36:21] "POST /epa/upload HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2024 16:36:21] "POST /epa/upload HTTP/1.1" 200 -


Pose detection failed.
Pose detection failed.
